#### Importação e seleção dos dados
Para treinar o modelo de predição dos placares, é necessário fazer uso de dados sobre os times assim como sobre as partidas, especificamente a quantidade de gols de cada time nas partidas. Por esse motivo, é utilizada a tabela unificada com dados das partidas e os times. Essa tabela atualmente se encontra como arquivo .csv, então é feito uso da biblioteca Pandas para importar ela como uma DataFrame usando a função Pandas.read_csv(). Nesse caso, a tabela é atribuída à variável "df".

In [18]:
import pandas as pd

df = pd.read_csv('newteams_with_matches.csv')

#### Seleção de alvo e features e treinamento dos modelos
Uma coisa que deve ser estabelecida é que serão necessários dois modelos diferentes, um pra prever os gols do time da casa e um para prever os gols do time visitante. Por isso, todas as etapas a partir de agora irão repetir para ambos modelos.
A tabela das partidas concatenada contém as informações dos times assim como dados coletados durante e após as partidas. Para que o modelo possa ser utilizado para prever partidas futuras, é importante ter cuidado para não fazer uso de dados coletados durante as partidas para o treinamento, pois esses dados são futuros, e não existem antes das partidas (exceto para o alvo, que no caso é o que o modelo irá prever).
Para aprimorar a acurácia do modelo, é possível criar novas colunas na tabela fazendo uso de dados já presentes nela. Ao chamar a variável da tabela (df) em uma coluna que ainda não existe nela, essa coluna irá ser criada com o valor que foi atribuído a ela, nesse caso criando uma coluna 'shots_on_target_percentage' (porcentagem de chutes ao gol) para cada time com os valores da divisão das colunas 'shots_on_target' (chutes ao gol) com as 'shots' (chutes totais).
Dito isso, essa célula é responsável pela seleção das colunas que serão utilizadas para o treinamento de cada modelo, assim como a criação de algumas colunas adicionais para aprimorar esse processo. Depois disso, essa célula também já realiza o treinamento de ambos modelos, assim como arquiva o modelo como arquivo Pickle, permitindo uso futuro do modelo sem retreinamento.

**Passo a Passo da Célula**

**1. Criação de colunas adicionais**

- **Descrição:** São criadas as colunas 'shots_on_target_percentage' (porcentagem de chutes ao gol) por meio da divisão entre as colunas 'shots_on_target' (chutes ao gol) e 'shots' (chutes totais).
- **Resultado:** As colunas '(home)_shots_on_target_percentage' e '(away)_shots_on_target_percentage' são criadas e inseridas dentro da dataframe principal 'df'

**2. Definição das features e alvo**

- **Descrição:** Cria uma lista com as features que serão utilizadas para cada modelo dentro da variável *features*, e define a variável *X* com elas. Também define a variável *y* como a coluna 'home_team_goal_count' e 'away_team_goal_count' para cada modelo, respectivamente.
- **Resultado:** As variáveis 'X_home' e 'X_away' são criadas como Pandas dataframes contendo apenas as características selecionadas de cada time, assim como as variáveis 'y_home' e 'y_away' com os dados dos gols de cada time nas partidas.

**3. Treinamento dos modelos**

- **Descrição:** Importa o RandomForestRegressor da biblioteca scikit-learn, define a função do modelo e seus hiperparâmetros em uma variável *model*. Depois faz uso da função *fit()* para treinar o modelo com as variáveis *X* e *y*. 
- **Resultado:** As variáveis 'model_home' e 'model_away' agora contém os modelos treinados, permitindo uso da função *predict()*.

**4. Salvando os modelos**

- **Descrição:** Importa a biblioteca Pickle para usar a função *dump()* e salvar os modelos em arquivos *.pkl*
- **Resultado:** São criados arquivos *placar_casa_randomforest.pkl* e *placar_visita_randomforest.pkl*.

In [19]:
# Criação de colunas com informações adicionais
df['(home)_shots_on_target_home_percentage'] = df['(home)_shots_on_target_home']/df['(home)_shots_home']
df['(away)_shots_on_target_away_percentage'] = df['(away)_shots_on_target_away']/df['(away)_shots_away']

# Seleção das colunas para o modelo dos gols do time da casa
features_home = ['(home)_minutes_per_goal_scored_home', '(away)_goals_conceded_per_match_away', 
                 '(home)_average_possession_home', '(home)_shots_on_target_home_percentage', 
                 '(home)_average_total_goals_per_match_home']
X_home = df[features_home]
y_home = df['home_team_goal_count']

# Seleção das colunas para o modelo dos gols do time visitante
features_away = ['(home)_average_total_goals_per_match_home', '(away)_minutes_per_goal_scored_away',
              '(home)_goals_conceded_per_match_home', '(away)_average_possession_away', 
              '(away)_shots_on_target_away_percentage', '(away)_average_total_goals_per_match_away']
X_away = df[features_away]
y_away = df['away_team_goal_count']

from sklearn.ensemble import RandomForestRegressor

# Para o modelo final, é utilizada a tabela inteira para treinamento, sem test split, para maior acurácia.
model_home = RandomForestRegressor(random_state=42, n_estimators=600, max_depth=15, max_leaf_nodes=10)
model_home.fit(X_home, y_home)

# Ditto.
model_away = RandomForestRegressor(random_state=42, n_estimators=600, max_depth=15, max_leaf_nodes=10)
model_away.fit(X_away, y_away)

import pickle

pickle.dump(model_home, open('placar_casa_randomforest.pkl', 'wb'))
pickle.dump(model_away, open('placar_visita_randomforest.pkl', 'wb'))

#### Utilização dos modelos
Com os modelos treinados, é necessário criar uma função que permita a predição de partidas futuras. Para isso, cria-se uma função que seja capaz de receber nomes de times, extrair os dados de cada time da tabela, formatar eles para estarem de acordo com as características no qual o modelo foi treinado, prever os resultado e retornar ele. É criada uma função para ambos modelos, que pode ser chamada em conjunto para predizer o resultado de um jogo específico.

**Passo a Passo da Célula**

**1. Extração dos dados da tabela**

- **Descrição:** Dentro de *prever_gols()* uma condicional *if* checa se os nomes dos times se encontra dentro da tabela. Caso sim, são selecionadas todas as características de cada time (função do Pandas *columns.str.contains()* que checa as colunas que contém '(home)' para o time da casa e ('away') para o time visitante).
- **Resultado:** Todas as características contidas na tabela do time da casa e visitante estão contidas nas variáveis locais 'stats_time_1' e 'stats_time_2' respectivamente, sendo elas dataframes de apenas uma linha.

**2. Preparação dos dados para predição**

- **Descrição:** Uma vez que as características de cada time estão contidas nas variáveis 'stats_time_1' e 'stats_time_2', e nenhuma delas está vazia (caso o nome de um dos times tenha sido digitado incorretamente), elas são então concatenadas em 'confronto_stats', onde a função do Pandas *reindex()* permite que sejam selecionadas apenas as características usadas no treinamento ao filtrar todas as colunas por 'modelo.feature_names_in'. Também há checagem de colunas duplicatas antes da predição.
- **Resultado:** A variável local 'confronto_stats' contém todas as características necessárias para predição, ordenadas corretamente.

**3. Predição dos gols do time**

- **Descrição:** Finalmente, é executada a função *predict()* do modelo com os dados de 'confronto_stats'. Os resultados são então arredondados para segunda ordem decimal mais próxima e finalmente são retornados pela função.
- **Resultado:** Ao executar a função *prever_gols_time()* com a tabela, nome dos times e modelo selecionado, retorna-se a quantidade de gols realizados pelo time. (ex.: ao executar a função com a tabela, 'Corinthians' como time da casa e 'Palmeiras' como time visitante, e o modelo que prevê gols do time da casa, retorna-se a predição da quantidade aproximada de gols que o Corinthians irá marcar nesse jogo.)

In [24]:
# Função para prever os gols do time da casa
def prever_gols(time_1, time_2, df, modelo):
    # Verifica se as colunas 'home_team_name' e 'away_team_name' estão no DataFrame
    if 'home_team_name' not in df.columns or 'away_team_name' not in df.columns:
        raise KeyError("As colunas 'home_team_name' ou 'away_team_name' estão faltando no DataFrame.")

    # O usuário não irá inputar todas as informações dos times, ele irá apenas selecionar dois dos times disponíveis.
    # É necessário então extrair as estatísticas de ambos times da tabela

    # Filtração das estatísticas do time da casa
    stats_time_1 = df[df['home_team_name'] == time_1].iloc[:, 32:]
    stats_time_1 = stats_time_1.loc[:, ~stats_time_1.columns.str.contains(r'\(away\)', case=False)]

    stats_time_1 = stats_time_1.iloc[0:1]  # Seleciona apenas a primeira linha

    # Filtração das estatísticas do time visitante
    stats_time_2 = df[df['away_team_name'] == time_2].iloc[:, 32:]
    stats_time_2 = stats_time_2.loc[:, ~stats_time_2.columns.str.contains(r'\(home\)', case=False)]

    stats_time_2 = stats_time_2.iloc[0:1]  # Seleciona apenas a primeira linha

    # Caso de erro
    if stats_time_1.empty or stats_time_2.empty:
        raise ValueError("Um dos times não foi encontrado no dataset.")

    # Concatena as estatísticas dos dois times
    confronto_stats = pd.concat([stats_time_1.reset_index(drop=True), stats_time_2.reset_index(drop=True)], axis=1)

    # Remover colunas duplicadas, se houver
    confronto_stats = confronto_stats.loc[:, ~confronto_stats.columns.duplicated()]

    # Garantir que as colunas estejam na mesma ordem que durante o treinamento
    if hasattr(modelo, 'feature_names_in_'):
        # Reordenar as colunas com base nas colunas usadas no treinamento do modelo
        confronto_stats = confronto_stats.reindex(columns=modelo.feature_names_in_, fill_value=0)

    # Previsão dos gols do time
    previsao_gols = modelo.predict(confronto_stats)

    # Arredondar os valores e garantir que não sejam negativos
    previsao_gols = max(round(previsao_gols[0], 2), 0)
    
    return previsao_gols

#### Célula de exemplo de execução dos modelos

- **Descrição:** Recebe-se os nomes de ambos times como inputs, que são usados para executar a função duas vezes, tanto para o time da casa quanto time visitante, retornando as predições dentro das variáveis 'gols_time_casa' e 'gols_time_visita'. Finalmente, esses valores podem ser usados para trazer a predição do placar final da partida, apenas arredondando eles para serem valores inteiros e os colocando em uma função *print()* do Python.
- **Resultado:** Ao executar a célula final e inserir os nomes dos times desejados, retorna-se a predição dos gols e placar final em texto no output da célula.

In [30]:
# Exemplo de uso da função para prever gols do time da casa
team_1 = input('Time da casa: ')
team_2 = input('Time visitante: ')

gols_time_casa = prever_gols(team_1, team_2, df, model_home)
gols_time_visita = prever_gols(team_1, team_2, df, model_away)

print(f"O time {team_1} (casa) marcará aproximadamente {gols_time_casa} gols contra o {team_2} (visitante), que marcará {gols_time_visita} gols.")
print(f"{team_1} {round(gols_time_casa)} x {round(gols_time_visita)} {team_2}")

O time Botafogo (casa) marcará aproximadamente 1.83 gols contra o Corinthians (visitante), que marcará 0.46 gols.
Botafogo 2 x 0 Corinthians
